In [ ]:
import pprint
from datetime import datetime
import requests
import json
import pandas as pd
import numpy as np
# add exception handling

In [ ]:
from datetime import date, timedelta

start_dt = date(2022, 6, 10)
end_dt = date(2022, 9, 15)

# difference between current and previous date
delta = timedelta(days=1)

# store the dates between two dates in a list
dates = []

while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
    dates.append(start_dt)
    # increment start date by timedelta
    start_dt += delta

print(dates)

[datetime.date(2022, 6, 10), datetime.date(2022, 6, 11), datetime.date(2022, 6, 12), datetime.date(2022, 6, 13), datetime.date(2022, 6, 14), datetime.date(2022, 6, 15), datetime.date(2022, 6, 16), datetime.date(2022, 6, 17), datetime.date(2022, 6, 18), datetime.date(2022, 6, 19), datetime.date(2022, 6, 20), datetime.date(2022, 6, 21), datetime.date(2022, 6, 22), datetime.date(2022, 6, 23), datetime.date(2022, 6, 24), datetime.date(2022, 6, 25), datetime.date(2022, 6, 26), datetime.date(2022, 6, 27), datetime.date(2022, 6, 28), datetime.date(2022, 6, 29), datetime.date(2022, 6, 30), datetime.date(2022, 7, 1), datetime.date(2022, 7, 2), datetime.date(2022, 7, 3), datetime.date(2022, 7, 4), datetime.date(2022, 7, 5), datetime.date(2022, 7, 6), datetime.date(2022, 7, 7), datetime.date(2022, 7, 8), datetime.date(2022, 7, 9), datetime.date(2022, 7, 10), datetime.date(2022, 7, 11), datetime.date(2022, 7, 12), datetime.date(2022, 7, 13), datetime.date(2022, 7, 14), datetime.date(2022, 7, 15), 

In [ ]:
def get_nearest_data(input_date, dates_list):
  cloz_dict = {
  abs(input_date.timestamp() - date.timestamp()) : date
  for date in dates_list}
  res = cloz_dict[min(cloz_dict.keys())]
  if(res>input_date):
    res = dates_list.index(res)+1
  else:
    res = dates_list.index(res)
  return res    

In [ ]:
# input date should be in any form. Make sure to correctly enter the values here 
def get_metrics(year1, month1, date1): # input should be date and ticker symbol
  input_date = datetime(year1, month1, date1) # (2023, 1, 21)
  url = "https://api-v2.intrinio.com/companies/GOOGL/filings?report_type=10-K,10-Q&api_key=OjAwYjk1OWYxZGEzYWQ2Y2VjN2YyMWExY2RkMTM2Y2Uz"
  ADVANCED_STATS = requests.get(url)
  op = ADVANCED_STATS.json()
  test_list = op['filings'] 
  res = [ sub['filing_date'] for sub in test_list ]
  dates_list = [datetime.strptime(date, '%Y-%m-%d') for date in res]
  item_of_use = get_nearest_data(input_date, dates_list)
  filing_code = op['filings'][item_of_use]['id']
  url2 = "https://api-v2.intrinio.com/filings/" + filing_code + "/fundamentals?api_key=OjAwYjk1OWYxZGEzYWQ2Y2VjN2YyMWExY2RkMTM2Y2Uz"
  filin = requests.get(url2)
  op2 = filin.json()
  # add exception handling
  # fix this. I cannot assume this.
  #pprint.pprint(len(op2['fundamentals']))
  income_statement_index = -1
  balance_sheet_index = -1
  cash_flow_index = -1
  for i in range(len(op2['fundamentals'])): 
    temp = op2['fundamentals'][i]
    # print(temp['statement_code']=="income_statement", temp['statement_code']=="balance_sheet_statement", temp['statement_code']=="cash_flow_statement")
    if(temp['statement_code']=='income_statement')and(income_statement_index == -1):
      income_statement_index = i
    if(temp['statement_code']=='cash_flow_statement')and(cash_flow_index == -1):
      cash_flow_index = i 
    if(temp['statement_code']=='balance_sheet_statement')and(balance_sheet_index == -1):
      balance_sheet_index = i
  #print(income_statement_index, balance_sheet_index, cash_flow_index)    
  list_of_statements = [op2['fundamentals'][income_statement_index]['id'], op2['fundamentals'][balance_sheet_index]['id'], op2['fundamentals'][cash_flow_index]['id']]
  tags = []
  value = []
  ops = []
  for i in list_of_statements:
    url3 = "https://api-v2.intrinio.com/fundamentals/" + i + "/standardized_financials?api_key=OjAwYjk1OWYxZGEzYWQ2Y2VjN2YyMWExY2RkMTM2Y2Uz"
    filin = requests.get(url3)
    op3 = filin.json()
    ops.append(op3)
    l = len(op3['standardized_financials'])
    for j in range(l): 
      temp = op3['standardized_financials'][j]['data_tag'] # these are sets of 3. But i need to visit each one of them and extract
      temp2 = op3['standardized_financials'][j]['value']
      tags.append(temp['tag'])
      value.append(temp2)
   #create df 
  return pd.DataFrame([value], columns = tags), ops

In [ ]:
df = get_metrics(2023, 1, 21)

# once i have op I need to check if all these statements are available. Only then can I call for them.

In [ ]:
df['weightedavedilutedsharesos'].values[0]

16325819000.0

In [ ]:
from tabulate import tabulate
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))

+----+--------------------+----------------+--------------------------+----------------------+--------------------+--------------+-------------+--------------------------+------------------------+---------------+--------------------+---------------------+--------------------+-----------------------+-------------+---------------------+----------------------------+------------+------------------------------+--------------+-----------------------------------+-------------------------+----------------------+------------------------+------------------+----------------------+----------------+----------------------+----------------------+------------+-----------------------+-------------------------+-------------------------+---------------+-----------------+-------------------+--------------------------+---------------------------+---------------------------+----------------+------------------------------+------------------------------+--------------------+-------------------------------+---

In [ ]:
df.to_csv('out.zip', index=False) 

In [ ]:
# netdebt = shorttermdebt + longtermdebt + capitalleaseobligations - cash - restrictedcash - fedfundssold - interestbearingdepositsatotherbanks - timedepositsplaced 
# where totaldebt is debt = shorttermdebt+longtermdebt+capitalleaseobligations

In [ ]:
def get_price_on_a_date(date, ticker_symbol):
  url = "https://cloud.iexapis.com/v1/data/CORE/HISTORICAL_PRICES/"+ticker_symbol+"?on="+date+"&token=sk_29735f4ddf4a47efb27623b229dda54a"
  HISTORICAL_PRICES = requests.get(url)
  op = HISTORICAL_PRICES.json()
  if len(op)!=0: # check to make sure we are not using a date when market was closed. 
  # Switch it with catch exception in function 
    close_price = op[0]["close"]
    return close_price
  else: 
    return 0
  return 0

In [ ]:
# get_price_on_a_date("2023-01-19", "AAPL")
import time
prices = []
#dates = dates[-30:]
for i in range(len(dates)):
  time.sleep(2)
  prices.append(get_price_on_a_date(dates[i], "AAPL"))


In [ ]:
# write code to to run throgh dates. 
# each df I get take sos and take net debt and put it into the df with recpective date. 
# Then iterate to next date. 
sos = []
nd = []
for i in dates:
  yr = i.year
  mon = i.month
  dt = i.day
  df_temp = get_metrics(yr, mon, dt)
  
  

In [ ]:
df, op = get_metrics(2023, 1, 21)

In [ ]:
df['shorttermdebt'].values[0]

21110000000.0

In [ ]:
# type(datetime(dates[0]))
# debt = shorttermdebt+longtermdebt+capitalleaseobligations
debt = df['shorttermdebt'].values[0]+ df['longtermdebt'].values[0] + df['capitalleaseobligations'].values[0] # if clo is not given consider is 0

In [ ]:
pprint.pprint(op[1])

{'fundamental': {'company': {'cik': '0001652044',
                             'id': 'com_dXOWOX',
                             'lei': None,
                             'name': 'Alphabet Inc',
                             'ticker': 'GOOGL'},
                 'end_date': '2022-09-30',
                 'filing_date': '2022-10-26T01:32:59.000Z',
                 'fiscal_period': 'Q3',
                 'fiscal_year': 2022,
                 'id': 'fun_B76rxA',
                 'is_latest': True,
                 'start_date': None,
                 'statement_code': 'balance_sheet_statement',
                 'type': 'reported',
                 'updated_date': '2023-03-20T04:04:24.132Z'},
 'standardized_financials': [{'data_tag': {'balance': 'debit',
                                           'factor': '+',
                                           'id': 'tag_NX6GGz',
                                           'name': 'Cash & Equivalents',
                                           'pare

In [ ]:
url3 = "https://api-v2.intrinio.com/fundamentals/" + "tag_NX6GGz" + "/standardized_financials?api_key=OjAwYjk1OWYxZGEzYWQ2Y2VjN2YyMWExY2RkMTM2Y2Uz"
filin = requests.get(url3)
op3 = filin.json()

In [ ]:
pprint.pprint(op3)

{'error': 'Invalid fundamental identifier',
 'message': 'An error occurred. Please contact success@intrinio.com with the '
            'details.'}
